In [ ]:
from IPython.display import display
import ipywidgets as widgets
from sentence_transformers import SentenceTransformer
from rapidfuzz import fuzz
import pandas as pd
import numpy as np
import faiss
import re
from dsci_utilities import BQHelper
from difflib import SequenceMatcher
import gc
from tqdm import tqdm

# GCP Configuration
GOOGLE_CLOUD_PROJECT = "mpb-data-science-dev-ab-602d"
GOOGLE_CLOUD_DATASET = "dsci_pricing_model"
GOOGLE_CLOUD_LOCATION = "us-central1"
GOOGLE_GENAI_USE_VERTEXAI = True
MODEL = "gemini-2.0-flash-001"

# BigQuery Helper
bq = BQHelper(
    billing_project_id=GOOGLE_CLOUD_PROJECT,
    write_project_id=GOOGLE_CLOUD_PROJECT,
    read_project_id=GOOGLE_CLOUD_PROJECT,
    write_dataset=GOOGLE_CLOUD_DATASET,
    read_dataset=GOOGLE_CLOUD_DATASET,
    daw_dataset=GOOGLE_CLOUD_DATASET,
    sql_folder="",
)

# Load data from BigQuery
df_in = bq.get_string(
    """
    SELECT
      ANY_VALUE(l.model_name) AS model_name,
      ANY_VALUE(l.primary_category) AS primary_category,
      ANY_VALUE(l.secondary_category) AS secondary_category,
      ANY_VALUE(l.product_type) AS product_type,
      ANY_VALUE(l.product_system) AS product_system,
      ANY_VALUE(l.brand) AS brand,
      ANY_VALUE(l.family) AS family,
      SUM(h.units_traded) / 5 AS units_traded
    FROM
      `mpb-data-science-dev-ab-602d.dsci_pricing_model.pm_base_050_model_lu` l
    JOIN
      `mpb-data-science-dev-ab-602d.dsci_pricing_model.pm_optimiser_040_scenarios_trade_history` h
    ON
      l.model_id = h.model_id
      AND l.market = h.market
    WHERE
      h.months_prior_to_current BETWEEN 1 AND 5
      AND h.sale_direction = 'buy'
    GROUP BY
      l.model_name
    """,
    silent=True,
)

# Prepare DataFrame
df = df_in.copy()
mem_usage_mb = df.memory_usage(deep=True).sum() / 1e6
print(f"DataFrame memory usage: {mem_usage_mb:.2f} MB")
n_rows = len(df)
embedding_dim = 384
embedding_bytes = n_rows * embedding_dim * 4  # float32 = 4 bytes
embedding_mb = embedding_bytes / 1e6

print(f"Estimated embedding matrix size: {embedding_mb:.2f} MB")

2025-05-19 10:08:23.582 | INFO     | dsci_utilities.BQHelper:get_string:328 - Running string: 
    SELECT
      ANY_VALUE(l.model_name) AS model_name,
      ANY_VALUE(l.primary_category) AS primary_category,
      ANY_VALUE(l.secondary_category) AS secondary_category,
      ANY_VALUE(l.product_type) AS product_type,
      ANY_VALUE(l.product_system) AS product_system,
      ANY_VALUE(l.brand) AS brand,
      ANY_VALUE(l.family) AS family,
      SUM(h.units_traded) / 5 AS units_traded
    FROM
      `mpb-data-science-dev-ab-602d.dsci_pricing_model.pm_base_050_model_lu` l
    JOIN
      `mpb-data-science-dev-ab-602d.dsci_pricing_model.pm_optimiser_040_scenarios_trade_history` h
    ON
      l.model_id = h.model_id
      AND l.market = h.market
    WHERE
      h.months_prior_to_current BETWEEN 1 AND 5
      AND h.sale_direction = 'buy'
    GROUP BY
      l.model_name
    
2025-05-19 10:08:26.142 | INFO     | dsci_utilities.BQHelper:_run_query:214 - 
	Duration: 0.37 seconds
	Data Processed

DataFrame memory usage: 0.52 MB
Estimated embedding matrix size: 1.53 MB


In [ ]:
# Define noise words
NOISE_WORDS = set(
    [
        "camera",
        "cameras",
        "cam",
        "product",
        "category",
        "brand",
        "family",
    ]
)


# Load resources
def load_resources():
    print("Loading dense encoder...")
    dense_model = SentenceTransformer("all-mpnet-base-v2", device="cpu")
    return dense_model


# Preprocessing
def preprocess_text(text):
    text = str(text).lower()
    return re.sub(r"[^a-z0-9\s]", "", text)


def build_search_blob(row, cols):
    text = " ".join(str(row[col]) for col in cols if pd.notnull(row[col]))
    text = re.sub(r"[^a-z0-9\s]", "", text.lower())
    tokens = text.split()

    seen = set()
    cleaned_tokens = []
    for token in tokens:
        if token and token not in seen and token not in NOISE_WORDS:
            seen.add(token)
            cleaned_tokens.append(token)

    return " ".join(cleaned_tokens)


# Build search dataset
def prepare_dataframe(df, cols):
    print("Generating search blobs and normalized fields...")
    df["search_blob"] = df.apply(lambda row: build_search_blob(row, cols), axis=1)
    df["units_traded"] = df["units_traded"].fillna(0)
    df["norm_popularity"] = np.log1p(df["units_traded"]) / np.log1p(
        df["units_traded"].max()
    )

    for col in ["model_name", "brand", "search_blob"]:
        df[f"{col}_normalized"] = df[col].apply(preprocess_text)

    df["embedding_input"] = (
        df["model_name_normalized"]
        + " "
        + df["brand_normalized"]
        + " "
        + df["search_blob_normalized"]
    )
    return df


# Generate embeddings
def generate_embeddings(texts, dense_model, batch_size=64):
    print("Generating dense embeddings...")
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i : i + batch_size]
        batch_embeds = dense_model.encode(batch, normalize_embeddings=True)
        embeddings.append(batch_embeds)
        del batch_embeds
        gc.collect()
    return np.vstack(embeddings)


# Build FAISS index
def build_faiss_index(embedding_matrix):
    print("Building FAISS index...")
    dim = embedding_matrix.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embedding_matrix)
    return index


# Search function
def search(query, df, index, dense_model, k=10, K=50):
    query_clean = preprocess_text(query)

    # If query is empty, return most popular products
    if not query_clean.strip():
        top_df = df.sort_values("norm_popularity", ascending=False).head(k).copy()
        return top_df.reset_index(drop=True)[
            ["model_name", "brand", "search_blob", "units_traded", "norm_popularity"]
        ].rename(
            columns={
                "model_name": "Model",
                "brand": "Brand",
                "search_blob": "Matched Text",
                "units_traded": "Monthly Sales",
                "norm_popularity": "Normalized Popularity",
            }
        )

    query_embedding = dense_model.encode(query_clean, normalize_embeddings=True)
    D, I = index.search(np.array([query_embedding]), k=K)  # noqa: E741
    results = []

    for idx, vec_score in zip(I[0], D[0]):
        row = df.iloc[idx]

        fuzzy_model = (
            fuzz.token_set_ratio(query_clean, row["model_name_normalized"]) / 100
        )
        fuzzy_brand = fuzz.token_set_ratio(query_clean, row["brand_normalized"]) / 100
        fuzzy_blob = (
            fuzz.token_set_ratio(query_clean, row["search_blob_normalized"]) / 100
        )
        char_sim = SequenceMatcher(
            None, query_clean, row["model_name_normalized"]
        ).ratio()
        exact_boost = (
            1.0
            if query_clean in row["model_name_normalized"]
            else (0.5 if query_clean in row["search_blob_normalized"] else 0.0)
        )

        hybrid_score = (
            0.4 * vec_score
            + 0.3 * fuzzy_model
            + 0.3 * fuzzy_brand
            + 0.2 * fuzzy_blob
            + 0.2 * char_sim
            + 0.5 * exact_boost
            + 0.4 * row["norm_popularity"]
        )

        results.append(
            {
                "Model": row["model_name"],
                "Brand": row["brand"],
                "Matched Text": row["search_blob_normalized"],
                "Model Name": row["model_name_normalized"],
                "Monthly Sales": row["units_traded"],
                "FAISS Score": vec_score,
                "Fuzzy Model": fuzzy_model,
                "Fuzzy Brand": fuzzy_brand,
                "Fuzzy Blob": fuzzy_blob,
                "Model Char Similarity": char_sim,
                "Exact Boost": exact_boost,
                "Normalized Popularity": row["norm_popularity"],
                "Hybrid Score": hybrid_score,
            }
        )

    return (
        pd.DataFrame(results)
        .sort_values("Hybrid Score", ascending=False)
        .reset_index(drop=True)[:k]
    )

In [3]:
# Load resources
dense_model = load_resources()

# Define columns used for blob generation
cols = [
    "model_name",
    "primary_category",
    "secondary_category",
    "product_type",
    "product_system",
    "brand",
    "family",
]

# Prepare data
df = prepare_dataframe(df, cols)

# Generate embeddings
embedding_matrix = generate_embeddings(df["embedding_input"].tolist(), dense_model)

# Build FAISS index
index = build_faiss_index(embedding_matrix)

print("Done.")

Loading dense encoder...
Generating search blobs and normalized fields...
Generating dense embeddings...


100%|██████████| 16/16 [00:13<00:00,  1.22it/s]

Building FAISS index...
Done.


In [4]:
# Search function wrapper
def search_query(query, k=10):
    return search(query, df, index, dense_model, k=k)

In [5]:
query_box = widgets.Text(
    placeholder="Search for products...",
    description="🔍 Query:",
    layout=widgets.Layout(width="80%"),
)

output = widgets.Output()


def on_query_submit(change):
    output.clear_output()
    with output:
        results = search_query(change["new"])
        display(results)


# Unregister previous observers to avoid duplicate triggers
query_box.unobserve_all()

# Register the observer
query_box.observe(on_query_submit, names="value")

display(query_box, output)

Text(value='', description='🔍 Query:', layout=Layout(width='80%'), placeholder='Search for products...')

Output()